In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import mysql.connector
from mysql.connector import Error
import pandas as pd
from datetime import datetime

#mysql
host = 'localhost'
database = 'buses'
user = 'root'
password = 'Dextral@11'
# Initialize the Chrome driver
driver = webdriver.Chrome()
driver.maximize_window()

try:
    # main URL
    driver.get('https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile')
    
    time.sleep(10) 

    # List of URLs to click sequentially
    urls = [
        'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode',
        'https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam',
        'https://www.redbus.in/bus-tickets/kozhikode-to-bangalore',
        'https://www.redbus.in/bus-tickets/ernakulam-to-kozhikode',
        'https://www.redbus.in/bus-tickets/kozhikode-to-mysore',
        'https://www.redbus.in/bus-tickets/kozhikode-to-thiruvananthapuram',
        'https://www.redbus.in/bus-tickets/bangalore-to-kalpetta',
        'https://www.redbus.in/bus-tickets/mysore-to-kozhikode',
        'https://www.redbus.in/bus-tickets/kalpetta-to-bangalore',
        'https://www.redbus.in/bus-tickets/kozhikode-to-thrissur',
        'https://www.redbus.in/bus-tickets/thiruvananthapuram-to-kozhikode',
        'https://www.redbus.in/bus-tickets/bangalore-to-kannur',
        'https://www.redbus.in/bus-tickets/kozhikode-to-kottayam',
        'https://www.redbus.in/bus-tickets/kannur-to-bangalore',
        'https://www.redbus.in/bus-tickets/kottayam-to-kozhikode',
        'https://www.redbus.in/bus-tickets/thrissur-to-kozhikode',
        'https://www.redbus.in/bus-tickets/kozhikode-to-kalpetta',
        'https://www.redbus.in/bus-tickets/coimbatore-to-ooty',
        'https://www.redbus.in/bus-tickets/kalpetta-to-kozhikode'  
    ]

    #data list
    bus_list = []

    # Loop through each URL
    for url in urls:
        # Get the URL
        driver.get(url)
        # Scroll down to load all bus elements
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(10)  
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Wait for the bus elements to load
        bus_elements = WebDriverWait(driver, 60).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//div[@class="clearfix row-one"]'))
        )
        
        # Extract bus information
        for bus_element in bus_elements:
            try:
                bus_info = {
                    'Operator': bus_element.find_element(By.XPATH, './/div[@class="travels lh-24 f-bold d-color"]').text,
                    'Type': bus_element.find_element(By.XPATH, './/div[@class="bus-type f-12 m-top-16 l-color evBus"]').text,
                    'Departure': bus_element.find_element(By.XPATH, './/div[@class="dp-time f-19 d-color f-bold"]').text,
                    'Duration': bus_element.find_element(By.XPATH, './/div[@class="dur l-color lh-24"]').text,
                    'Arrival': bus_element.find_element(By.XPATH, './/div[@class="bp-time f-19 d-color disp-Inline"]').text,
                    'Date': bus_element.find_element(By.XPATH, './/div[@class="next-day-dp-lbl m-top-16"]').text,
                    'From': bus_element.find_element(By.XPATH, './/div[@class="dp-loc l-color w-wrap f-12 m-top-42"]').text,
                    'To': bus_element.find_element(By.XPATH, './/div[@class="bp-loc l-color w-wrap f-12 m-top-8"]').text,
                    'Rating': bus_element.find_element(By.XPATH, './/div[@class="rating-sec lh-24"]').text,
                    'Price': bus_element.find_element(By.XPATH, './/div[@class="fare d-block"]').text,
                    'Seats Available': bus_element.find_element(By.XPATH, './/div[@class="seat-left m-top-30"]').text,
                }
                bus_list.append(bus_info)
            except Exception as e:
                print(f"Exception occurred while extracting bus info: {str(e)}")
                continue

    # Creating DataFrame from bus_list
    df_buses = pd.DataFrame(bus_list)

    print(df_buses)

except Exception as e:
    print(f"Exception occurred: {str(e)}")
    import traceback
    print(traceback.format_exc())

finally:
    driver.quit()
    
try:
    # Connect to MySQL
    connection = mysql.connector.connect(host=host,
                                         database=database,
                                         user=user,
                                         password=password)
    
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print(f"Connected to MySQL Server version {db_Info}")
        
        # Create a cursor object using cursor() method
        cursor = connection.cursor()

        # Create table if not exists
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS buses_data (
            id INT AUTO_INCREMENT PRIMARY KEY,
            Operator VARCHAR(255),
            Type VARCHAR(255),
            Departure VARCHAR(20),
            Duration VARCHAR(20),
            Arrival VARCHAR(20),
            Date DATE,  -- Change type to DATE
            FromLocation VARCHAR(255),
            ToLocation VARCHAR(255),
            Rating VARCHAR(10),
            Price VARCHAR(20),
            SeatsAvailable VARCHAR(50)
        )
        '''
        cursor.execute(create_table_query)
        print("Table 'buses_data' created or already exists")

        df_buses = pd.DataFrame(bus_list) 

        # Insert into mysql
        for index, row in df_buses.iterrows():
            # Convert '23-Jul' to 'YYYY-MM-DD'
            date_str = row['Date'] + '-2024' 
            date_obj = datetime.strptime(date_str, '%d-%b-%Y').date() 
            data_to_insert = (row['Operator'], row['Type'], row['Departure'], row['Duration'], 
                              row['Arrival'], date_obj, row['From'], row['To'], 
                              row['Rating'], row['Price'], row['Seats Available'])
            
            insert_query = '''
            INSERT INTO buses_data (Operator, Type, Departure, Duration, Arrival, Date, FromLocation, ToLocation, Rating, Price, SeatsAvailable)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''
            cursor.execute(insert_query, data_to_insert)
            connection.commit()
        
        print("Data inserted into 'buses_data' table successfully")

except Error as e:
    print(f"Error while connecting to MySQL: {e}")

finally:
    # Close cursor and connection
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


Exception occurred while extracting bus info: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@class="next-day-dp-lbl m-top-16"]"}
  (Session info: chrome=127.0.6533.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF775029632+30946]
	(No symbol) [0x00007FF774FDE3C9]
	(No symbol) [0x00007FF774ED6FDA]
	(No symbol) [0x00007FF774F2822C]
	(No symbol) [0x00007FF774F2850C]
	(No symbol) [0x00007FF774F1BCDC]
	(No symbol) [0x00007FF774F4CAAF]
	(No symbol) [0x00007FF774F1BBB6]
	(No symbol) [0x00007FF774F4CC80]
	(No symbol) [0x00007FF774F6B041]
	(No symbol) [0x00007FF774F4C813]
	(No symbol) [0x00007FF774F1A6E5]
	(No symbol) [0x00007FF774F1B021]
	GetHandleVerifier [0x00007FF77515F83D+1301229]
	GetHandleVerifier [0x00007FF77516BDB7+1351783]
	GetHandleVerifier [0x00007FF775162A03+1313971]
	GetHandleVerifier [0x00007FF7750